In [64]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import pandas as pd

In [65]:
# Path to chromedriver
#get_ipython().system('which chromedriver')

# Set the executable path and initialize the chrome browser in spliter
executable_path = {'executable_path': 'C:\Program Files (x86)\chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [66]:
# ### Visit the NASA Mars News Site
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [67]:
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [68]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

In [62]:
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [69]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8963/nasas-perseverance-rover-begins-its-first-science-campaign-on-mars/" target="_self">NASA's Perseverance Rover Begins Its First Science Campaign on Mars</a></div>